In [1]:
import latentvelo as ltv
import numpy as np
import scanpy as sc
import scvelo as scv
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import anndata as ad

SEED = 2024
np.random.seed(SEED)

Global seed set to 0


In [2]:
adata = sc.read_h5ad("../adata/redeem_young.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 9144 × 2000
    obs: 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'ATAC.weight', 'seurat_clusters', 'Sig.HSC1', 'Sig.Prog1', 'Sig.EarlyE1', 'Sig.LateE1', 'Sig.ProMono1', 'Sig.Mono1', 'Sig.ncMono1', 'Sig.cDC1', 'Sig.pDC1', 'Sig.ProB1', 'Sig.PreB1', 'Sig.B1', 'Sig.Plasma1', 'Sig.T1', 'Sig.CTL1', 'Sig.NK1', 'STD.CellType', 'STD_Cat', 'STD_Cat2', 'Sample', 'MitoCoverage', 'ClonalGroup', 'ClonalGroup.Prob', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'CellType', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'velocity_self_transition'
    var: 'name', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'CellType_colors', 'STD.CellType_colors', 'neighbors', 'umap', 'velocity_gra

In [3]:
# adata = ltv.utils.standard_clean_recipe(adata,normalize_library = False)
adata = ltv.utils.standard_clean_recipe(adata)
adata.var['velocity_genes'] = True

spliced_key = 'spliced'
unspliced_key = 'unspliced'

spliced_library_sizes = adata.layers[spliced_key].sum(1)
unspliced_library_sizes = adata.layers[unspliced_key].sum(1)

if len(spliced_library_sizes.shape) == 1:
       spliced_library_sizes = spliced_library_sizes[:,None]
if len(unspliced_library_sizes.shape) == 1:
       unspliced_library_sizes = unspliced_library_sizes[:,None]

adata.obs['spliced_size_factor'] = spliced_library_sizes #spliced_all_size_factors
adata.obs['unspliced_size_factor'] = unspliced_library_sizes #unspliced_all_size_factors

model = ltv.models.VAE(observed = 2000) # observed: number of genes
epochs, val_ae, val_traj = ltv.train(model, adata,name="scNT")


Extracted 2000 highly variable genes.
Choosing top 2000 genes
computing neighbors
    finished (0:00:19) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:01) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
Warning, folder already exists. This may overwrite a previous fit.
2000 velocity genes used
epoch 0, full loss 126.575, val loss 39.342, recon MSE 0.393, traj MSE 0.037, reg loss -1.684
epoch 1, full loss 74.392, val loss 10.719, recon MSE 0.317, traj MSE 0.033, reg loss -2.080
epoch 2, full loss 74.423, val loss -15.012, recon MSE 0.213, traj MSE 0.032, reg loss -2.316
epoch 3, full loss 101.309, val loss -40.589, recon MSE 0.115, traj MSE 0.032, reg loss -2.305
epoch 4, full loss 150.505, val loss -64.644, recon MSE 0.074, traj MSE 0.033, reg loss -2.598
epoch 5, full loss 225.367, val loss -85.742, recon MSE 0.069, traj MSE 0.032, reg loss -2.522
epoc

In [4]:
latent_adata, adata = ltv.output_results(model, adata, gene_velocity=True,
                                         embedding='umap')


In [5]:
print(adata)
print(latent_adata)

AnnData object with n_obs × n_vars = 9144 × 2000
    obs: 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'ATAC.weight', 'seurat_clusters', 'Sig.HSC1', 'Sig.Prog1', 'Sig.EarlyE1', 'Sig.LateE1', 'Sig.ProMono1', 'Sig.Mono1', 'Sig.ncMono1', 'Sig.cDC1', 'Sig.pDC1', 'Sig.ProB1', 'Sig.PreB1', 'Sig.B1', 'Sig.Plasma1', 'Sig.T1', 'Sig.CTL1', 'Sig.NK1', 'STD.CellType', 'STD_Cat', 'STD_Cat2', 'Sample', 'MitoCoverage', 'ClonalGroup', 'ClonalGroup.Prob', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'CellType', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'velocity_self_transition', 'spliced_size_factor', 'unspliced_size_factor', 'batch_id', 'celltype_id', 'root', 'terminal', 'latent_time'
    var: 'name', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'veloci

In [6]:
# adata.write_h5ad("../adata/latentvelo.h5ad")
# latent_adata.write_h5ad("../adata/latentvelo_latent.h5ad")